In [ ]:
import os
import gc
import numpy as np
import pandas as pd
from scipy.stats import kurtosis
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
import xgboost as xgb
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
DATA_DIRECTORY = "../input/home-credit-loan-better-data-processing"

In [ ]:
train = pd.read_csv(os.path.join(DATA_DIRECTORY, 'train.csv'))
test_df = pd.read_csv(os.path.join(DATA_DIRECTORY, 'test.csv'))
labels = pd.read_csv(os.path.join(DATA_DIRECTORY, 'labels.csv'))

In [ ]:
import re
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test = test_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
labels = labels.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
train=np.nan_to_num(train)
test=np.nan_to_num(test)
labels=np.nan_to_num(labels)

In [ ]:
train = pd.DataFrame(train)
test = pd.DataFrame(test)
labels=pd.DataFrame(labels)

In [ ]:
train.head()

In [ ]:
print(test.shape)

# Enasamble-1(LGBM+RF+NN)

# LGBM

In [ ]:
from lightgbm import LGBMClassifier

lgbmc = LGBMClassifier()
lgbmc.fit(train, labels)

In [ ]:
lgbm_pred = lgbmc.predict_proba(test)[:,1]

In [ ]:
lgbm_pred_train = lgbmc.predict_proba(train)[:,1]

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf =RandomForestClassifier()
rf.fit(train, labels)

In [ ]:
rf_pred = rf.predict_proba(test)[:,1]

In [ ]:
rf_pred_train = rf.predict_proba(train)[:,1]

In [ ]:
lgbm_pred=pd.DataFrame(lgbm_pred)
rf_pred=pd.DataFrame(rf_pred)

In [ ]:
lgbm_pred_train=pd.DataFrame(lgbm_pred_train)
rf_pred_train=pd.DataFrame(rf_pred_train)

In [ ]:
df_test = pd.concat([lgbm_pred, rf_pred], axis=1)
predd=lgbm_pred
del lgbm_pred, rf_pred
gc.collect()

In [ ]:
df_test.head()

In [ ]:
df_train = pd.concat([lgbm_pred_train, rf_pred_train], axis=1)
del lgbm_pred_train, rf_pred_train
gc.collect()

In [ ]:
df_train.head()

In [ ]:
print(df_train.shape)

# NN

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential([
    Dense(1000, activation='relu', input_shape=(2,)),
    Dense(1000, activation='relu'),
    Dense(1000, activation='relu'),
    Dense(1000, activation='relu'),
    Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
hist = model.fit(df_train, labels,
          batch_size=32, epochs=2)

In [ ]:
pred = model.predict_proba(df_test)

In [ ]:
submit = test_df[['SK_ID_CURR']]
submit['TARGET'] = predd
submit.to_csv('ensamble_1_updated_data.csv', index = False)